In [ ]:
import numpy as np
import graphinglib as gl
import pyregion
from scipy.optimize import curve_fit
import scipy
import dill
from functools import partial

from src.tools.statistics.advanced_stats import autocorrelation_function, autocorrelation_function_2d, \
                                                get_autocorrelation_function_2d_contour
from src.hdu.maps.map import Map

In [ ]:
def get_plottables(data: np.ndarray, region_radius: float) -> list:
    sorted_data = data[np.argsort(data[:,0])]
    scatter = gl.Scatter(
        sorted_data[:,0],
        sorted_data[:,1],
        marker_size=5,
        face_color="black",
    )

    # scatter.add_errorbars(
    #     y_error=sorted_data[:,2],
    #     errorbars_line_width=0.25,
    #     cap_width=0,
    # )

    return [scatter]

In [ ]:
turbulence = Map.load("summer_2023/gaussian_fitting/maps/computed_data_selective/turbulence.fits")

regions = [
    ("Global region", None, 50),
    ("Diffuse region", pyregion.open("summer_2023/gaussian_fitting/regions/region_1.reg"), 20),
    ("Central region", pyregion.open("summer_2023/gaussian_fitting/regions/region_2.reg"), 10),
    ("Filament region", pyregion.open("summer_2023/gaussian_fitting/regions/region_3.reg"), 6)
]

### Autocorrelation functions of all regions

In [ ]:
figs = []

for name, region, region_radius in regions:
    data = autocorrelation_function(turbulence.get_masked_region(region).data, method="Boily")

    fig = gl.Figure(
        title=name,
        x_label="Lag",
        y_label="Autocorrelation function",
        x_lim=(0, region_radius*1.2),
        y_lim=(-1, 1),
    )
    plottables = get_plottables(data, region_radius)
    cropped_scatter = plottables[0].create_slice_x(0, region_radius*1.2)
    # fig.y_lim = np.min(cropped_scatter.y_data)-0.2, np.max(cropped_scatter.y_data)+0.3
    fig.add_elements(*plottables)
    figs.append(fig)

multifig = gl.MultiFigure.from_grid(figs, (2,2), (14, 9))
multifig.save("figures/sh158/advanced_stats/autocorrelation/autocorrelation_functions_new_2.pdf")

### 2D Autocorrelation function

In [ ]:
data = autocorrelation_function_2d(turbulence.data, method="Boily")
fig = gl.Figure(
    x_lim=(np.min(data[:,0]), np.max(data[:,0])),
    y_lim=(-np.max(data[:,1]), np.max(data[:,1])),
)
cont = get_autocorrelation_function_2d_contour(data)

fig.add_elements(cont)
fig.save("test_new.pdf")

# Tests

In [ ]:
d = np.array([
    [1,2,3],
    [4,5,6],
    [7,8,9]
])
print(autocorrelation_function(d))

d_mean = np.array([
    [-4,-3,-2],
    [-1, 0, 1],
    [ 2, 3, 4]
])
print(np.sum(d_mean**2))

In [ ]:
-4*-3+-3*-2+-4*-1+-3*0+-2*1+-1*0+0*1+-1*2+0*3+1*4+2*3+3*4 # lag = 1

In [ ]:
40/60*np.sqrt(9)/(12)**(3/2) / (np.sqrt(9)/(9)**(3/2))

## Test python script

In [ ]:
np.random.seed(3)
test_array = np.random.random((50,50))
test_array[:,1] = np.NAN
# print(test_array - np.nanmean(test_array))
acr = autocorrelation_function(test_array, "Boily")
# print(acr)

In [ ]:
new_array = test_array - np.nanmean(test_array)

# Create arrays that will be useful for computing distances
x, y = np.arange(new_array.shape[1]), np.arange(new_array.shape[0])
xx, yy = np.meshgrid(x, y)
dists_and_multiplications = []

for y in range(new_array.shape[0]):
    for x in range(new_array.shape[1]):
        if not np.isnan(new_array[y, x]):
            multiplication = new_array[y, x] * new_array
            dists = np.sqrt((x-xx)**2 + (y-yy)**2)
            # The multiplication's result is linked to the pixels' distance
            dists_and_multiplications.append(np.stack((dists, multiplication), axis=2))

unique_dists = {}
for pix in dists_and_multiplications:
    for row in pix:
        for dist, multiplication in row:
            unique_dists[dist] = (unique_dists.get(dist, []) + [multiplication])

mean_dists = []
norm = np.nanmean(new_array**2)
for dist, vals in unique_dists.items():
    mean_dists.append([dist, np.nanmean(vals) / norm])
arr = np.array(mean_dists)
arr = arr[np.argsort(arr[:,0])]
print(arr)

In [ ]:
print(np.allclose(acr[:,:2], arr))